In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import numpy as np
import seaborn
import pandas as pd
import os
import faiss
import numpy as np
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
from typing import List

2024-12-09 20:55:44.225661: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
os.getcwd()

'/Users/michaelaeichel/Documents/GitHub/INFO4940_RAG_Project'

In [3]:
model_name = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/opt/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
embeddings_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [5]:
class LLMWrapper:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def invoke(self, prompt: str) -> str:
        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
        outputs = self.model.generate(
            **inputs,
            max_length=100,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=self.tokenizer.eos_token_id
        )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)
    def generate(self, query: str) -> str:
        augmented_query = f"Question: {query}\nAnswer:"
        inputs = self.tokenizer(augmented_query, return_tensors="pt", max_length=512, truncation=True)
        outputs = self.model.generate(
            inputs["input_ids"],
            max_length=100,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=self.tokenizer.eos_token_id
        )
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

In [6]:
llm = LLMWrapper(model, tokenizer)

In [42]:
query = "What is the function of the liver in the human body?"
response = llm.generate(query)
print(response)

Question: What is the function of the liver in the human body?
Answer: In the liver, the liver is responsible for the functions of the cells, their function is to provide energy and the activity of the cell. It is located in the liver, which is responsible for the functions of the cells, their function is to provide energy. When the liver is not functioning, the cells are not functioning properly due to the toxicity of the substances in the liver.
In the liver, the liver
